# Jupyter GUI for ryven

## Import and setup ryven

TODO: clean up and polish

In [1]:
import ryvencore as rc
import json
import os

In [2]:
from ryven.main.utils import import_nodes_package, load_from_file

os.environ['RYVEN_MODE'] = 'no-gui'

In [3]:
from os.path import basename, dirname, splitext, normpath, join

class NodesPackage:
    """
    A small container to store meta data about imported node packages.
    """

    def __init__(self, directory: str):

        self.name = basename(normpath(directory))
        self.directory = directory

        self.file_path = normpath(join(directory, 'nodes.py'))

    def config_data(self):
        return {
            'name': self.name,
            'dir': self.directory,
        }

In [4]:
import ryven.NENV as NENV
NENV.init_node_env()

In [5]:
# from ryven.std.special_nodes import nodes as special_nodes

In [6]:
packages = ['ryven/std', 'ryven/nodes/built_in/']

In [7]:
session = rc.Session()
for package in packages:
    session.register_nodes(
        import_nodes_package(
            NodesPackage(
                directory=package 
            )
        )
    )

In [8]:
# session = rc.Session()
# session.register_nodes(
#     import_nodes_package(
#         NodesPackage(
#             directory='ryven/std' 
#         )
#     )
# )
# session.nodes[0].identifier

In [9]:
# session.register_nodes(
#     import_nodes_package(
#         NodesPackage(
#             directory='ryven/nodes/built_in/'
#         )
#     )
# )
# # session.nodes

### Load original ryven workflow

In [10]:
fpath = 'basics.json'
with open(fpath, 'r') as f:
        project: dict = json.loads(f.read())

In [11]:
session.load(project);

In [12]:
script0 = session.scripts[0]
node0 = script0.flow.nodes[0]
node0.identifier

'std.Storage_Node'

In [13]:
for n in script0.flow.nodes:
    print (n.identifier, n.GLOBAL_ID)

std.Storage_Node 7
std.Eval_Node 10
built_in.Result_Node 13
std.Slider_Node 15
built_in.Result_Node 19
built_in.SetVar_Node 21
built_in.GetVar_Node 25
std.Clock_Node 28
built_in.SetVar_Node 32
built_in.GetVar_Node 38
std.Plus_Node 41
std.LinkIN_Node 45


In [14]:
for c in script0.flow.connections:
    print (c.inp.node.GLOBAL_ID, c.out.node.GLOBAL_ID)

21 7
13 10
19 15
45 15
10 25
32 28
7 38
41 38
32 41


In [15]:
input0 = node0.inputs[0]
input0.label_str

''

In [16]:
node0 = script0.flow.nodes[0]
len(script0.flow.connections), len(script0.flow.nodes)

(9, 12)

#### Remove node example

Note: Requires still removal/update of connections

In [17]:
# script0.flow.remove_node(node0)
# script0.flow.nodes

In [18]:
for c in script0.flow.connections:
# con0 = script0.flow.connections[0]
    print (c.out.node.GLOBAL_ID, c.inp.node.GLOBAL_ID)

7 21
10 13
15 19
15 45
25 10
28 32
38 7
38 41
41 32


#### Load information rearding gui directly from json

In [19]:
import json
 
# Opening JSON file
with open('basics.json') as f:
    data = json.load(f)

In [20]:
for n in data['scripts'][0]['flow']['nodes']:
    name = n['identifier'].split('.')[-1].split('_')[0]
    x = n['pos x']
    y = n['pos y']
    print (name, x, y)


Storage 475.4795814751692 264.55248052403067
Eval 462.86363383615935 370.17323807221123
Result 796.5680046166863 369.6301042263301
Slider 273.19774517706185 503.4372312306748
Result 571.2515010942456 555.0662156323955
SetVar 724.2936205842011 249.9898748171978
GetVar 164.80323966981427 368.23084671814206
Clock 231.23622828581165 114.6139516414438
SetVar 846.1910218628586 133.30644603168525
GetVar 196.31117068512344 265.7809649154535
Plus 585.6887913914879 174.51150293252385
LinkIN 566.7044488232787 478.94789784811906


## Jupyter GUI

In [21]:
from ipycanvas import Canvas, hold_canvas
import numpy as np

import ipywidgets as widgets

In [22]:
class canvas_widget:
    def __init__(self, gui=None, width=2000, height=1000, script=None):
        self.gui = gui
        self.script = script
        self._width, self._height = width, height

        self._col_background = "black" # "#584f4e"
        self._col_node_header = "blue" #"#38a8a4"
        self._col_node_selected = "#9dcea6"
        self._col_node_unselected = "#dee7bc"
        
        self._font_size = 30
        self._node_box_size = 160, 70

        self._canvas = Canvas(width=width, height=height)
        self._canvas.fill_style = self._col_background
        self._canvas.fill_rect(0, 0, width, height)
        self._canvas.layout.width = '100%'
        self._canvas.layout.height = 'auto'

        self.objects_to_draw = []
        self.connections = []
        
        self._canvas.on_mouse_down(self.handle_mouse_down)
        self._canvas.on_mouse_move(self.handle_mouse_move)        
        
    def draw_connection(self, path=[0, 1]):
        i_out, i_in = path
        out = self.objects_to_draw[i_out]
        inp = self.objects_to_draw[i_in]

        canvas = self._canvas
        canvas.stroke_style = 'white' 
        canvas.line_width = 3
        canvas.move_to(out.x_out, out.y_out)
        canvas.line_to(inp.x_in, inp.y_in)
        canvas.stroke()
    
    def canvas_restart(self):
        self._canvas.clear()
        self._canvas.fill_style = self._col_background
        self._canvas.fill_rect(0, 0, self._width, self._height)

    def handle_mouse_down(self, x, y):
        if [o for o in self.objects_to_draw if o.selected]:
            [o.set_selected(False) for o in self.objects_to_draw if o.selected]
            self.redraw()
            return False

        check_bool_pos = list(
            set([check_region.is_selected(x, y) for check_region in self.objects_to_draw])
        )
        if self.gui is None:
            title = 'None'
        else:
            title = self.gui.nodes.value
            
        if len(check_bool_pos) == 1:
            if check_bool_pos[0]:
                self.canvas_restart()
            else:
                self.add_node(x, y, self.script.session.nodes[1])
#             if not check_bool_pos[0]:
#                 self.add_node(x, y, self.script.session.nodes[1])
#             else:
#                 self.canvas_restart()

        if len(check_bool_pos) == 0:
            self.add_node(x, y, self.script.session.nodes[1])
            
        self.redraw()

    def handle_mouse_move(self, x, y):
        if [o for o in self.objects_to_draw if o.selected]:
            with hold_canvas(self._canvas):
                [o for o in self.objects_to_draw if o.selected][-1].set_x_y(x, y)
                self.canvas_restart()
                [o.draw() for o in self.objects_to_draw]
                for path in self.connections:
                    self.draw_connection(path)
                    
    def redraw(self):
        self.canvas_restart()
        with hold_canvas(self._canvas):
            self.canvas_restart()
            [o.draw() for o in self.objects_to_draw]
            for path in self.connections:
                self.draw_connection(path)  
          
    def _read_script(self):
        node_gid_dic = {}
        nodes = self.script.flow.nodes
#         nodes_gui = data['scripts'][i_script]['flow']['nodes']
        for i_n, n in enumerate(nodes):
            name = n.identifier.split('.')[-1].split('_')[0]
            o = self.objects_to_draw[i_n]
            x = o.x
            y = o.y

#             n = nodes_gui[i_n]
            gid = n.GLOBAL_ID
            node_gid_dic[gid] = i_n  # TODO: check why indices in json-file are shifted by 2

            val = ''
            if hasattr(n, 'val'):
                val = n.val

            s = NodeGui(x//0.6, y//0.6, self, title=name, val=val)
            s.set_selected(False)

        for c in self.script.flow.connections:
            c_in = c.inp.node.GLOBAL_ID
            c_out = c.out.node.GLOBAL_ID
            con = [node_gid_dic[c_out], node_gid_dic[c_in]]
            self.connections.append(con)                
                    
    def add_node(self, x, y, node):
        n = self.script.flow.create_node(node)
        print ('node: ', n.identifier, n.GLOBAL_ID)
        name = n.identifier.split('.')[-1].split('_')[0]
        s = NodeGui(x, y, self, title=name)
        s.set_selected(False)
        self.objects_to_draw.append(s)

        self.redraw()     
    
    def delete_selected(self):
        for o in self.objects_to_draw:
            if o.selected:
                self.objects_to_draw.remove(o)
        #TODO: remove connections        
        self.redraw()        
    

In [23]:
class NodeGui:
    def __init__(self, x, y, canvas_widget, selected=False, title='Title', val=None):
        self.x = x
        self.y = y
        self.width, self.height = canvas_widget._node_box_size
        self.selected = selected
        self.title = title
        self.val = val
        self.canvas_widget = canvas_widget
        self.canvas = canvas_widget._canvas
#         self.canvas_widget.objects_to_draw.append(self)
#         n_obj = len(self.canvas_widget.objects_to_draw)
#         if n_obj > 1:
#             self.canvas_widget.connections.append([n_obj-2, n_obj-1])

    def set_x_y(self, x_in, y_in):
        self.x = x_in
        self.y = y_in
        
    def draw_title(self, title):
        self.canvas.fill_style = self.canvas_widget._col_background
        self.canvas.font = f'{self.canvas_widget._font_size}px serif'
        self.canvas.fill_style = 'white'
        x = self.x - (self.width * 0.49)
        y = self.y - (self.height * 0.6),
        self.canvas.fill_text(title, x, y)
        
    def draw_value(self, val):
        self.canvas.fill_style = self.canvas_widget._col_background
        self.canvas.font = f'{self.canvas_widget._font_size}px serif'
        x = self.x - (self.width * 0.4)
        y = self.y - (self.height * 0.01),
        self.canvas.fill_text(val, x, y)        
        
    def draw_input(self):
        self.canvas.fill_style = self.canvas_widget._col_background
        x = self.x - (self.width * 0.5)
        y = self.y - 0 * (self.height * 0.5)
        self.x_in, self.y_in = x, y        
        self.canvas.fill_style = 'blue'
        self.canvas.fill_arc(x, y, 5, -np.pi/2, np.pi/2)      
        
    def draw_output(self):
        self.canvas.fill_style = self.canvas_widget._col_background
        x = self.x + (self.width * 0.5)
        y = self.y - 0 * (self.height * 0.5)
        self.x_out, self.y_out = x, y
        self.canvas.fill_style = 'red'
        self.canvas.fill_arc(x, y, 5, np.pi/2, 3 * np.pi/2)        

    def draw(self):
        self.canvas.fill_style = self.canvas_widget._col_node_header
        self.canvas.fill_rect(
            self.x - (self.width * 0.5), self.y - (self.height), self.width, self.height
        )
        if self.selected:
            self.canvas.fill_style = self.canvas_widget._col_node_selected
        else:
            self.canvas.fill_style = self.canvas_widget._col_node_unselected
        self.canvas.fill_rect(
            self.x - (self.width * 0.5),
            self.y - (self.height * 0.5),
            self.width,
            self.height,
        )
        self.draw_title(self.title)
        if self.val:
            self.draw_value(self.val)
        self.draw_input()
        self.draw_output()
        

    def is_selected(self, x_in, y_in):
        x_coord = self.x - (self.width * 0.5)
        y_coord = self.y - (self.height * 0.5)

        if (
            x_in > x_coord
            and x_in < (x_coord + self.width)
            and y_in > y_coord
            and y_in < (y_coord + self.height)
        ):

            self.set_selected(True)
            return True
        else:
            self.set_selected(False)
            return False

    def set_selected(self, state):
        self.selected = state

In [24]:
ng = NodeGui(1, 1, cw)

NameError: name 'cw' is not defined

In [25]:
ng.draw_value??

Object `ng.draw_value` not found.


In [26]:
session = rc.Session()
for package in packages:
    session.register_nodes(
        import_nodes_package(
            NodesPackage(
                directory=package 
            )
        )
    )
    
session.create_script(title='test')    

In [27]:
onto_dic = {}
onto_dic['Utilities'] = {'Murnaghan':[]}
onto_dic['Codes'] = {'Vasp':[], 'Lammps':[]}
onto_dic['Structures'] = {'Bulk':[], 'Surfaces':[]}

gui_modes = ['Add Node', 'Delete Node', 'Add Connection', 'None']

class GUI:
    def __init__(self, onto_dic=onto_dic):
        session = rc.Session()
        for package in packages:
            session.register_nodes(
                import_nodes_package(
                    NodesPackage(
                        directory=package 
                    )
                )
            )
    
        script = session.create_script(title='test') 
        
        self.canvas_widget = canvas_widget(self, script=script)
        self.onto_dic = onto_dic
        
    def draw(self):    
            out = widgets.Output(layout={'border': '1px solid black'})
            with out:
                display(self.canvas_widget._canvas)

            self.modules = widgets.Dropdown(
                options=self.onto_dic.keys(),
                value='Codes',
            #     description='Category:',
                disabled=False,
                layout=widgets.Layout(width='100px')
            )

            self.mode = widgets.Dropdown(
                options=gui_modes,
                value=gui_modes[0],
            #     description='Category:',
                disabled=False,
                layout=widgets.Layout(width='100px')
            )        

            self.nodes = widgets.RadioButtons(
                options=self.onto_dic[self.modules.value].keys(),
            #    value='pineapple', # Defaults to 'pineapple'
            #    layout={'width': 'max-content'}, # If the items' names are long
            #     description='Nodes:',
                disabled=False
            )

    #         run_button = widgets.Button(description="Run")
    #         add_node_button = widgets.Button(description="Add Node")
    #         add_connection_button = widgets.Button(description="Add Connection")

            self.modules.observe(self.on_value_change, names='value')

            return widgets.VBox([
                          widgets.HBox([self.mode]), 
                          widgets.HBox([
                              widgets.VBox([self.modules, self.nodes]), 
                              out])
                           ])     


    def on_value_change(self, change):
        self.nodes.options = onto_dic[self.modules.value].keys()

In [28]:
gui = GUI()
gui.draw()

In [27]:
cw = canvas_widget(script=session.scripts[0])
cw._canvas

Canvas(height=1000, layout=Layout(height='auto', width='100%'), width=2000)

In [ ]:
cw._read_script()
cw.redraw()

In [ ]:
%%time
i_script = 0

node_gid_dic = {}
nodes = session.scripts[i_script].flow.nodes
nodes_gui = data['scripts'][i_script]['flow']['nodes']
for i_n, n in enumerate(nodes_gui):
    name = n['identifier'].split('.')[-1].split('_')[0]
    x = n['pos x']
    y = n['pos y']

    n = nodes[i_n]
    gid = n.GLOBAL_ID
    node_gid_dic[gid] = i_n  # TODO: check why indices in json-file are shifted by 2

    val = ''
    if hasattr(n, 'val'):
        val = n.val
        
    s = NodeGui(x//0.6, y//0.6, cw, title=name, val=val)
    s.set_selected(False)
    
for c in script0.flow.connections:
    c_in = c.inp.node.GLOBAL_ID
    c_out = c.out.node.GLOBAL_ID
    con = [node_gid_dic[c_out], node_gid_dic[c_in]]
    cw.connections.append(con)

cw.redraw()

In [29]:
n = session.nodes[0]

In [ ]:
cw.script.session.nodes[10].identifier.split('.')[-1]

In [30]:
cw.add_node(300, 400, n)

node:  std.std.std.Checkpoint_Node 97


NameError: name 'NodeGui' is not defined

In [ ]:
cw.handle_mouse_down(1,1)

In [ ]:
cw.script.flow.nodes  #[0].GLOBAL_ID

In [ ]:
n3.set_state({'val': 6}, 0)
n3.update_event()
cw.redraw()

In [ ]:
script0.flow.a

In [29]:
for n in script0.flow.nodes:
    if hasattr(n, 'val'):
        print (n.identifier, n.GLOBAL_ID, n.val)

built_in.Result_Node 13 3134
std.std.std.Slider_Node 15 0.533
built_in.Result_Node 19 1.066


In [30]:
n3 = script0.flow.nodes[3]
n3.input(0)
# n3.update_event()
# n3.set_output_val(0,3)

2

In [35]:
n3.inputs[0].node

In [ ]:
n2 = script0.flow.nodes[2]
n2.actions

In [ ]:
n7 = script0.flow.nodes[7]

In [ ]:
n7.actions

In [ ]:
n7.start()

In [ ]:
n7.stop()

In [ ]:
o7 = n7.outputs[0]
o7.get_val()

In [ ]:
n3.outputs[0].get_val()

In [ ]:
i3 = n3.inputs[0]
i3.val = 1

In [ ]:
n3.actions

In [ ]:
n3.set_output_val(0, 19)
n3.update_event()

In [ ]:
n2.input(0)

In [ ]:
n3.inputs[0].val = 2
n3.set_state({'val': 8}, 0)
n3.update_event()
n3.outputs[0].val, script0.flow.nodes[4].val

In [ ]:
script0.flow.nodes[4].val

In [ ]:
n7.outputs[0].connected()

In [ ]:
# o3.connections[0].out.val, o3.connections[0].inp.val

In [ ]:
# o3.connections[0].inp.node

In [ ]:
script0.flow.nodes[2]

In [ ]:
script0.flow.nodes[3].GLOBAL_ID, script0.flow.nodes[4].GLOBAL_ID

In [ ]:
script0.flow.nodes[3].outputs[0].connections[0].inp.node.GLOBAL_ID

In [ ]:
script0.flow.nodes[4].val

In [ ]:
script0.flow.nodes[3].outputs[0].connections[0].inp.node

In [ ]:
for o in cw.objects_to_draw:
    if o.selected:
        cw.objects_to_draw.remove(o)

In [ ]:
cw.delete_selected()

In [ ]:
o.selected = True
cw.redraw()

In [ ]:
n = nodes[0]


In [ ]:
session.all_node_objects()

In [ ]:
script0.flow.add_node(session.all_node_objects()[1])

In [ ]:
script0.flow.

In [ ]:
script0.flow.remove_node(script0.flow.nodes[-1])

In [ ]:
script0.flow.create_nodes_from_data??

In [ ]:
from ryvencore.utils import node_from_identifier

In [ ]:
# script0.flow.create_node()

In [ ]:
node_from_identifier('std.Button_Node', session.nodes)

In [ ]:
n3 = script0.flow.nodes[3]
n3.identifier, n3.GLOBAL_ID, 

In [ ]:
session.register_node??

In [ ]:
aa = session.nodes[1]
aa.identifier

In [ ]:
session.nodes

In [ ]:
script0.session.nodes